In [287]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

## Import the Data

In [288]:
def load_data():
    
    import os
    import cv2
    
    root = r"/home/feketrob/Demosaiking/generated_training_data/"
    
    train_images = []
    train_truth = []
    test_images = []
    test_truth = []
    
    train_set = 0
    test_set = 0
    train_to_test = 0.8
    
    for folder in sorted(os.listdir(root)):
        
        path = os.path.join(root, folder)
        images = sorted(os.listdir(path))
        current_index = 0
        
        while current_index < len(images):
            image_paths = images[current_index: current_index + 3]
            image_b = cv2.imread(os.path.join(path, image_paths[0]), 0)
            image_g = cv2.imread(os.path.join(path, image_paths[1]), 0)
            image_r = cv2.imread(os.path.join(path, image_paths[2]), 0)
            image = np.zeros((image_b.shape[0], image_b.shape[1], 3))
            image = np.array([image_r, image_g, image_b])
            
            truth_paths = images[current_index + 3: current_index + 6]
            truth_b = cv2.imread(os.path.join(path, truth_paths[0]), 0)
            truth_g = cv2.imread(os.path.join(path, truth_paths[1]), 0)
            truth_r = cv2.imread(os.path.join(path, truth_paths[2]), 0)
            truth = np.zeros((truth_b.shape[0], truth_b.shape[1], 3))
            truth = np.array([truth_r, truth_g, truth_b])
            
            if train_set > train_to_test * (train_set + test_set):
                test_images.append(image)
                test_truth.append(truth)
                test_set += 1
            else:
                train_images.append(image)
                train_truth.append(truth)
                train_set += 1
            
            current_index += 6
            
    return ((np.asarray(train_images), np.asarray(train_truth)), (np.asarray(test_images), np.asarray(test_truth)))
    

In [289]:
#import data

((train_images, train_truth), (test_images, test_truth)) = load_data()

## Explore the Data

In [290]:
print('Training data shape : ', train_images.shape, train_truth.shape)

print('Testing data shape : ', test_images.shape, test_truth.shape)


Training data shape :  (3360, 3, 33, 33) (3360, 3, 21, 21)
Testing data shape :  (840, 3, 33, 33) (840, 3, 21, 21)


## Preprocess the Data

In [291]:
# Find the shape of input images and create the variable input_shape
nDims, nRows,nCols = train_images.shape[1:]
input_shape = (nRows, nCols, nDims)
train_data = train_images
test_data = test_images
train_data = train_images.reshape(train_images.shape[0], nRows, nCols, nDims)
test_data = test_images.reshape(test_images.shape[0], nRows, nCols, nDims)

print(train_data.shape)

nDims, nRows,nCols = train_truth.shape[1:]
train_truth_data = train_truth
test_truth_data = test_truth
train_truth_data = train_truth.reshape(train_truth.shape[0], nRows, nCols, nDims)
test_truth_data = test_truth.reshape(test_truth.shape[0], nRows, nCols, nDims)
print(train_truth_data.shape)

# Change to float datatype
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_truth_data = train_truth_data.astype('float32')
test_truth_data = test_truth_data.astype('float32')

# Scale the data to lie between 0 to 1
train_data /= 255
test_data /= 255
train_truth_data /= 255
test_truth_data /=255

(3360, 33, 33, 3)
(3360, 21, 21, 3)


## Define the model

In [ ]:
def gkern(kernlen=9, nsig=3):
    """Returns a 2D Gaussian kernel array."""

    import scipy.stats as st
    
    interval = (2*nsig+1.)/(kernlen)
    x = np.linspace(-nsig-interval/2., nsig+interval/2., kernlen+1)
    kern1d = np.diff(st.norm.cdf(x))
    kernel_raw = np.sqrt(np.outer(kern1d, kern1d))
    kernel = kernel_raw/kernel_raw.sum()
    return kernel


def my_init(shape, dtype=None):
    
    print(shape)
    return gkern(shape[0])

def createModel():
    model = Sequential()
    
    model.add(Conv2D(128, (9, 9), activation='relu', input_shape=input_shape,
                    ))
    
    model.add(Conv2D(64, (1, 1), activation='relu'))
    model.add(Conv2D(3, (5, 5), activation='relu'))
    
    return model

## Train the model

In [ ]:
model1 = createModel()
batch_size = 256
epochs = 50
model1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model1.summary()

history = model1.fit(train_data, train_truth_data, batch_size=batch_size, epochs=epochs, verbose=1, 
                   validation_data=(test_data, test_truth_data))
model1.evaluate(test_data, test_truth_data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 25, 25, 128)       31232     
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 25, 25, 64)        8256      
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 21, 21, 3)         4803      
Total params: 44,291
Trainable params: 44,291
Non-trainable params: 0
_________________________________________________________________
Train on 3360 samples, validate on 840 samples
Epoch 1/50
3360/3360 [==============================] - 71s 21ms/step - loss: nan - acc: 0.4277 - val_loss: nan - val_acc: 0.4451
Epoch 2/50
3360/3360 [==============================] - 70s 21ms/step - loss: nan - acc: 0.4431 - val_loss: nan - val_acc: 0.4451
Epoch 3/50
3360/3360 [==============================] - 69s 21ms/step - loss: nan - acc: 0.4431

## Check the loss and accuracy curves

In [ ]:
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

## Save model

In [ ]:
# Save model
import os
import uuid

models_rep = r"/home/feketrob/Demosaiking/models"
folder_name = str(uuid.uuid1())
target_folder = os.path.join(models_rep, folder_name)
os.mkdir(target_folder)

model_json = model.to_json()
with open(os.path.join(target_folder, "model.json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(target_folder, "model.h5"))
print("Saved model to disk")

print(target_folder)